In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv(r"C:\Imp\SamanyuK\Datasets\ad_10000records.csv")
df

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
col=['Ad Topic Line','City','Country','Timestamp']
df.drop(columns=col,axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['Gender']=df['Gender'].apply(lambda x: 0 if x=='Female' else 1)

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [ ]:
x=df.drop(columns='Clicked on Ad')
y=df['Clicked on Ad']
x_t,x_te,y_t,y_te=train_test_split(x,y,test_size=0.25,random_state=20)

In [ ]:
dt=DecisionTreeClassifier()
path=dt.cost_complexity_pruning_path(x_t,y_t)
alphas=path.ccp_alphas

In [ ]:
param={'ccp_alpha':alphas}
ndt=RandomizedSearchCV(dt,param_distributions=param,cv=10,scoring='accuracy',n_jobs=-1,n_iter=100)
ndt.fit(x_t,y_t)
print(ndt.best_score_)
print(ndt.best_params_)
ba=ndt.best_params_['ccp_alpha']

In [ ]:
dt=DecisionTreeClassifier(ccp_alpha=ba)
param={'criterion':['gini','entropy'], 'max_depth':list(np.arange(1,201)), 'min_samples_split':list(np.arange(1,201)),'min_samples_leaf':list(np.arange(1,201)),
      'max_features':['sqrt','log2',None]}
ndt=RandomizedSearchCV(dt,param_distributions=param,scoring='accuracy',cv=10,n_jobs=-1,n_iter=100)
ndt.fit(x_t,y_t)
print(ndt.best_score_)
print(ndt.best_params_)

In [ ]:
pred=ndt.predict(x_te)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
cr=classification_report(y_te,pred)
a=accuracy_score(y_te,pred)
cm=confusion_matrix(y_te,pred)
print(cr,a)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15,6))
sns.heatmap(cm,annot=True)
plt.show()